<a href="https://colab.research.google.com/github/hypro2/hands-on-LLM-from-colab/blob/main/Setfit_Sbert_Few_Shot_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate peft setfit

In [ ]:
!pip install transformers==4.42.2

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")
model.model_head

LogisticRegression()

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2", use_differentiable_head=True, head_params={"out_features": 5, 'temperature': 0.5,})
model.model_head

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


SetFitHead({'in_features': 768, 'out_features': 5, 'temperature': 0.5, 'bias': True, 'device': 'cuda'})

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# 1. CSV 파일 읽기
train_df = pd.read_csv("/content/drive/MyDrive/05_sentence.csv")
eval_df = pd.read_csv("/content/drive/MyDrive/05_sentence_eval.csv")
train_df.head()

,skey,ti,ab,clr,cla,ipcs,label_idx
0,4920053002728,waste to energy conversion without co2 emissions,the invention provides a method for energy ext...,1 a process for obtaining energy from waste wh...,1 a process for obtaining energy from waste wh...,C10J C01B,0
1,4920052001746,polyimide film for graphite sheet graphite she...,the present disclosure provides a polyimide fi...,1 a polyimide film prepared from a precursor c...,1 a polyimide film prepared from a precursor c...,C01B C09K C08J C08G,0
2,4920052001747,reaction furnace for producing polycrystalline...,a reaction furnace for producing a polycrystal...,1 a reaction furnace for producing a polycryst...,1 a reaction furnace for producing a polycryst...,C01B C23C B01J C23C,0
3,4920051002143,compositions and processes for removing heavy ...,processes for removingrecovering heavy metal i...,1 a process for removing heavy metal ions from...,1 a process for removing heavy metal ions from...,C01B,0
4,4920051002144,method of making nanomaterials from a renewabl...,this patent disclosure includes a process that...,1 a process for making carbon nanosheet materi...,1 a process for making carbon nanosheet materi...,C01B B82Y C01B,0


In [ ]:
train_df["text"] = train_df[["ti", "ab","cla"]].fillna("").agg(" ".join, axis=1)
eval_df["text"] = eval_df[["ti", "ab", "cla"]].fillna("").agg(" ".join, axis=1)

train_df = train_df[["skey", "text", "label_idx"]].rename(columns={"label_idx": "label"})
eval_df = eval_df[["skey", "text", "label_idx"]].rename(columns={"label_idx": "label"})

In [ ]:
train_df['label'] = train_df['label'].astype('int')
eval_df['label'] = eval_df['label'].astype('int')

In [ ]:
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(eval_df)
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['skey', 'text', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['skey', 'text', 'label'],
        num_rows: 200
    })
})


In [ ]:
from setfit import sample_dataset

train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=10) # 라벨당 샘플 8개만 훈련
train_dataset

Dataset({
    features: ['skey', 'text', 'label'],
    num_rows: 50
})

In [ ]:
test_dataset = dataset["test"]
test_dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dataset({
    features: ['skey', 'text', 'label'],
    num_rows: 200
})

In [ ]:
model.labels = ["CPC_C01B", "CPC_C01C", "CPC_C01D", "CPC_C01F", "CPC_C01G"]

In [ ]:
from setfit import Trainer
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=16,
    num_epochs=3,
    body_learning_rate = 7e-6,
    eval_strategy = "epoch",
    eval_max_steps = 50
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 2000
  Batch size = 16
  Num epochs = 3
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hyeong9647 (hyeong9647-personal). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.124900,0.215324
2,0.005400,0.215672
3,0.002200,0.216786


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 384.


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
trainer.evaluate(test_dataset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running evaluation *****


{'accuracy': 0.625}

In [ ]:
trainer.evaluate(test_dataset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running evaluation *****


{'accuracy': 0.665}

In [ ]:
# model.save_pretrained("setfit-bge-small-v1.5-cpcc-8-shot")

In [ ]:
preds = model.predict([
    "provided is a method of producing a zeolite film continuously and efficiently the method of forming zeolite on a surface of a support is characterized in that the method includes a first step of attaching zeolite fine crystals to a surface of a support a second step of preparing synthetic gel for growing the fine crystals a third step of putting the support and the synthetic gel into a continuous reactor and performing hydrothermal synthesis and a fourth step of cleaning the support on which zeolite has been hydrothermally synthesized and in the third step the temperature pressure and flow of the synthetic gel in the continuous reactor is adjusted the support is moved being immersed in the synthetic gel the reaction time of the hydrothermal synthesis is adjusted by adjusting the time from when the support enters the continuous reactor to when the support exits the continuous reactor",
    "a method for producing metal chloride mxclx includes reacting metal carbonate in solid form using phosgene diphosgene andor triphosgene to form metal chloride mxclx wherein the metal m is selected from the group containing alkali metals alkaline earth metals al and zn li and mg or li for example and x corresponds to the valency of the metal cations an apparatus for performing such method is also disclosed",
    "the present application relates to a metal oxide and synthesis of a lithium ion battery specifically the present application selects a cobalt oxide compound which uses co3o4 as a main body as a precursor of lithium cobalt oxide and anion doping is performed in particles of co3o4 to obtain a doped precursor for lithium cobalt oxide the general formula of the precursor can be expressed as co3o1ymy4 where about 0ltyltabout 02 and wherein the anion m comprises at least one of f p s cl n as se br te i or at the lithium ion battery with a cathode made of lithium cobalt oxide material prepared by using the precursor presents good cycle stability in a high voltage chargedischarge environment",
])
preds

['CPC_C01B', 'CPC_C01F', 'CPC_C01D']

In [ ]:
!pip install optuna

In [ ]:
from datasets import load_dataset  # 데이터셋 로드 함수
from setfit import SetFitModel, Trainer, sample_dataset  # SetFit 모델 및 관련 클래스/함수
from optuna import Trial  # Optuna의 Trial 객체 (단일 실험 정의)
from typing import Dict, Union, Any  # 타입 힌트를 위한 유틸리티

# 모델 초기화 함수 정의
def model_init(params: Dict[str, Any]) -> SetFitModel:
    params = params or {}  # 전달된 파라미터가 없을 경우 빈 딕셔너리 사용
    max_iter = params.get("max_iter", 100)  # 최대 반복 횟수 (기본값 100)
    solver = params.get("solver", "liblinear")  # 로지스틱 회귀 솔버 (기본값 'liblinear')
    params = {
        "head_params": {  # 로지스틱 회귀 파라미터
            "max_iter": max_iter,
            "solver": solver,
        }
    }
    # 사전 학습된 문장 임베딩 모델 로드 및 설정 적용
    return SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2", **params)

# 하이퍼파라미터 탐색 공간 정의
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "body_learning_rate": trial.suggest_float("body_learning_rate", 5e-6, 2e-5, log=True),
        # 모델 본체의 학습률 (5e-6 ~ 2e-5 범위에서 로그 스케일로 샘플링)
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),  # 학습 에폭 수 (주석 처리됨)
        # "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64]),  # 배치 크기 (주석 처리됨)
        # "seed": trial.suggest_int("seed", 1, 40),  # 랜덤 시드 값 (주석 처리됨)
        # "max_iter": trial.suggest_int("max_iter", 50, 300),  # 최대 반복 횟수 (주석 처리됨)
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
        # 로지스틱 회귀 솔버 선택 (newton-cg, lbfgs, liblinear 중)
    }

# Trainer 객체 초기화
trainer = Trainer(
    train_dataset=train_dataset,  # 학습 데이터셋 (외부에서 정의되어야 함)
    eval_dataset=test_dataset,  # 평가 데이터셋 (외부에서 정의되어야 함)
    model_init=model_init,  # 초기화 함수 참조
)

# 하이퍼파라미터 탐색 수행
best_run = trainer.hyperparameter_search(
    direction="maximize",  # 평가 지표를 최대화하는 방향으로 탐색
    hp_space=hp_space,  # 하이퍼파라미터 탐색 공간
    n_trials=10  # 10번의 탐색 수행
)
print(best_run)  # 가장 성능이 좋은 하이퍼파라미터 출력

# 최적의 하이퍼파라미터 적용
trainer.apply_hyperparameters(
    best_run.hyperparameters,  # 최적화된 하이퍼파라미터 사용
    final_model=True  # 최종 모델로 설정
)

# 모델 학습
trainer.train()

# 모델 평가
metrics = trainer.evaluate()
print(metrics)  # 평가 지표 출력


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

[I 2024-11-28 05:17:08,546] A new study created in memory with name: no-name-c1308f1d-0306-42d2-9674-9c66b2904052
Trial: {'body_learning_rate': 8.591450423524714e-06, 'solver': 'liblinear'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 2000
  Batch size = 16
  Num epochs = 1
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hyeong9647 (hyeong9647-personal). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,0.308100
50,0.193600
100,0.202500


***** Running evaluation *****
[I 2024-11-28 05:22:38,633] Trial 0 finished with value: 0.66 and parameters: {'body_learning_rate': 8.591450423524714e-06, 'solver': 'liblinear'}. Best is trial 0 with value: 0.66.
Trial: {'body_learning_rate': 1.2604066795876683e-05, 'solver': 'newton-cg'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 2000
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.308100
50,0.193600
100,0.202500


***** Running evaluation *****
[I 2024-11-28 05:28:03,587] Trial 1 finished with value: 0.68 and parameters: {'body_learning_rate': 1.2604066795876683e-05, 'solver': 'newton-cg'}. Best is trial 1 with value: 0.68.
Trial: {'body_learning_rate': 1.876670960784095e-05, 'solver': 'newton-cg'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 2000
  Batch size = 16
  Num epochs = 1
[W 2024-11-28 05:28:07,115] Trial 2 failed with parameters: {'body_learning_rate': 1.876670960784095e-05, 'solver': 'newton-cg'} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 65.06 MiB is free. Process 249671 has 14.68 GiB memory in use. Of the allocated memory 14.43 GiB is allocated by PyTorch, and 122.69 MiB is reserved by Py

OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 65.06 MiB is free. Process 249671 has 14.68 GiB memory in use. Of the allocated memory 14.43 GiB is allocated by PyTorch, and 122.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)